# Calendar API

Create a function that takes a list of events and returns the earliest available time where all people can attend.

In [75]:
from typing import List, Tuple
from collections import defaultdict

class MyCalendar:
    """Calendar class as an API template
    """
    def __init__(self,times):
        """initialize the calendar with the given times

        Args:
            times (_type_): dictionary with persons as keys and their busy time in a day from 0-24 as values
        """
        self.times = times
        self.available_times = self.get_earliest_available_time()
    
    def decimal2mins(self,time:float)->int: 
        """convert decimal time in hours to minutes

        Args:
            time (float): _description_

        Returns:
            int: _description_
        """
        hrs = int(time) 
        mins = int((time-hrs)*60) 
        t = hrs*60 + mins 
        return t

    def mins2decimal(self,time:int,decimals:int = 2)->float: 
        """convert minutes into decimal time in hours

        Args:
            time (int): _description_
            decimals (int, optional): _description_. Defaults to 2.

        Returns:
            float: _description_
        """
        hrs = time//60 
        mins = (time%60)/60 
        return round(hrs+mins,2)

    def str2time(self,start:float,end:float) -> List[int]:
        """convert start and end time in decimal format to minutes

        Args:
            start (float): _description_
            end (float): _description_

        Returns:
            List[int]: _description_
        """
        return [self.decimal2mins(start),self.decimal2mins(end)]
    
    def merge_intervals(self,intervals:List[List[float]]):
        merged_intervals = []
        sorted_intervals = sorted(intervals,key=lambda x:x[0])
        for interval in sorted_intervals:
            if not merged_intervals or merged_intervals[-1][1] < interval[0]:
                merged_intervals.append(interval)
            else:
                merged_intervals[-1][1] = max(merged_intervals[-1][1],interval[1])
        return merged_intervals

    def check_earliest(self,busy:List[List]):
        if not busy or not busy[0]:
            return (float('inf'),True)
        earliest = float('inf')
        busy = self.merge_intervals(intervals=busy)
        is_start_zero = busy[0][0] == 0
        if not is_start_zero:
            earliest = 0
            return (earliest,is_start_zero) 
        for times in busy:
            start,end = times[0],times[1]
            if start == 0:
                is_start_zero = True
            if end <= earliest:
                earliest = end
        return (earliest,is_start_zero) 

    def get_earliest_available_time(self) -> float:
        """given a list of persons busy time, find the earliest available time

        Returns:
            float: _description_
        """
        earliest_available_times = defaultdict(float)
        zeros = [] 
        for person , time_slots in self.times.items(): 
            earliest, is_start_zero = self.check_earliest(time_slots) 
            print(f"For person {person} earliest available time is {earliest}")
            earliest_available_times[person] = earliest 
            zeros.append(is_start_zero) 
        earliest_available = (max(earliest_available_times.values()) if any(zeros) else 0) 
        return earliest_available

times = {
    0: [[0, 6.57], [9, 10],[0.20,2]], 
    1: [[0, 6], [11, 12]],
    2: [[0,6.43]]
}
cal = MyCalendar(times=times)
print(cal.available_times)
times = {
    0: [[1, 6.57], [9, 10],[1.20,2]], 
    1: [[1, 6], [11, 12]],
    2: [[0,6.66]]
}
cal = MyCalendar(times=times)
print(cal.available_times)










For person 0 earliest available time is 6.57
For person 1 earliest available time is 6
For person 2 earliest available time is 6.43
6.57
For person 0 earliest available time is 0
For person 1 earliest available time is 0
For person 2 earliest available time is 6.66
6.66


In [3]:
!pip install mimesis

In [50]:
from mimesis import Datetime, Person
from mimesis.locales import Locale
from datetime import datetime, timedelta, time
from collections import defaultdict
import random
from typing import List, Dict, Optional
import json

class SceduleGenerator:
    def __init__(self, start_year=2020, end_year=2020, n_persons=2, n_days=7, min_activites=1, max_activites=4, locale=Locale.EN):
        self.locale = locale
        self.date_gen = Datetime(locale=locale)
        self.person_gen = Person(locale=locale)
        self.start_year = start_year
        self.end_year = end_year
        self.n_persons = n_persons
        self.n_days = n_days
        self.min_activites = min_activites
        self.max_activites = max_activites
        self.schedules = self.generate_weekly_schedule(start_year=self.start_year,end_year=self.end_year, n_persons=self.n_persons,n_days=self.n_days,min_activites=self.min_activites,max_activites=self.max_activites)

    def generate_weekly_schedule(self,start_year=2020, end_year=2020,n_persons=2,n_days=7,min_activites=1,max_activites=4,min_duration=30,max_duration=180):
        date_gen = Datetime(locale=Locale.EN)
        person_gen = Person(Locale.EN)
        start_datetime = date_gen.datetime(start=start_year,end=end_year)
        schedules = defaultdict(list)
        for n_persons in range(n_persons):
            person_name = person_gen.full_name()
            for day in range(n_days):
                current_date = start_datetime + timedelta(days=day)
                
                # Generate 2-4 busy time slots for each day
                daily_schedule = []
                for _ in range(random.randint(min_activites, max_activites)):
                    start_time = date_gen.time()
                    duration = timedelta(minutes=random.randint(min_duration, max_duration))  # 30 minutes to 3 hours
                    end_time = (datetime.combine(current_date, start_time) + duration).time()
                    
                    activity = person_gen.occupation() # Generate a random activity (using occupation as an example)
                    
                    # Create datetime objects for start and end times
                    start_dt = datetime.combine(current_date, start_time)
                    end_dt = start_dt + duration  # Add duration to start_datetime
                    daily_schedule.append({
                        'start': start_dt,
                        'end': end_dt,
                        'activity': activity
                    })
                    
                
                # Sort the daily schedule by start time
                daily_schedule.sort(key=lambda x: x['start'])
                daily_schedule = self.merge_intervals(daily_schedule)
                
                schedules[person_name].append({
                    'date': current_date.date(),
                    'activities': daily_schedule
                })
            
        return schedules


    def jsonify_schedule(self,schedules:List[Dict]):
        return json.dumps(schedules,indent=4)

    def time_to_minutes(self,t:datetime):
        return t.hour * 60 + t.minute

    def minutes_to_time(self,m):
        hours, minutes = divmod(m, 60)
        return time(hours % 24, minutes)

    def merge_intervals(self,intervals):
        # Sort the intervals based on the start time
        intervals_sorted = sorted(intervals, key=lambda x: x['start'])

        merged = []
        for interval in intervals_sorted:
            # If merged is empty or if current interval doesn't overlap with the previous one
            if not merged or merged[-1]['end'] < interval['start']:
                merged.append(interval)
            else:
                # Update the end time of the previous interval if necessary
                merged[-1]['end'] = max(merged[-1]['end'], interval['end'])

        return merged

schedules = SceduleGenerator(n_days=2).schedules

# Print the schedule
for k,v in schedules.items():
    print(f"\n{k}")
    for day in v:
        print(f"{day['date']}")
        for activity in day['activities']:
            print(f"  {activity['start']} - {activity['end']} : {activity['activity']}")


Frank Mcintyre
2020-07-19
  2020-07-19 08:42:05.387013 - 2020-07-19 10:46:05.387013 : Gallery Owner
  2020-07-19 14:50:34.292454 - 2020-07-19 16:17:34.292454 : Anaesthetist
2020-07-20
  2020-07-20 10:10:40.612609 - 2020-07-20 12:51:40.612609 : Miner
  2020-07-20 15:28:20.518881 - 2020-07-20 15:58:20.518881 : Tax Inspector
  2020-07-20 22:18:58.576999 - 2020-07-21 00:37:58.576999 : Stockman

Cherlyn Scott
2020-07-19
  2020-07-19 05:25:42.692899 - 2020-07-19 09:48:56.924502 : Editor
  2020-07-19 13:14:44.644294 - 2020-07-19 15:33:44.644294 : Recorder
2020-07-20
  2020-07-20 17:44:28.616553 - 2020-07-20 19:36:28.616553 : Hotelier
  2020-07-20 22:41:52.442297 - 2020-07-21 00:55:52.442297 : Youth Worker


In [67]:

from datetime import date
import pandas as pd
import plotly.express as px

class MyCal:
    """Calendar API base class for computing schedules for a set of persons
    """
    def __init__(self,schedules:dict):
        self.schedules = schedules
        self.persons = list(self.schedules.keys())
        # self.print_schedules()
        
    def merge_intervals(self,intervals):
        # Sort the intervals based on the start time
        intervals_sorted = sorted(intervals, key=lambda x: x['start'])

        merged = []
        for interval in intervals_sorted:
            # If merged is empty or if current interval doesn't overlap with the previous one
            if not merged or merged[-1]['end'] < interval['start']:
                merged.append(interval)
            else:
                # Update the end time of the previous interval if necessary
                merged[-1]['end'] = max(merged[-1]['end'], interval['end'])

        return merged
        
    def print_schedules(self):
        for k,v in self.schedules.items():
            print(f"\n{k}")
            for day in v:
                print(f"{day['date']}")
                for activity in day['activities']:
                    print(f"  {activity['start']} - {activity['end']}")
        
    def check_conflicts(self):
        NotImplemented
    
    def get_earliest_availability(self, person:str):
        earliest_availability_of_person_by_date = defaultdict(dict)
        # # print(person,schedules[person])
        for schedule in self.schedules[person]:
            curr_date = schedule['date']
            # print(person,curr_date)
            earliest_availability_of_person_by_date[person][curr_date] = datetime.combine(date.max, datetime.max.time())
            for activity in schedule['activities']:
                if activity['end'] <= earliest_availability_of_person_by_date[person][curr_date]:
                    earliest_availability_of_person_by_date[person][curr_date] = activity['end']
                    
        for k,v in earliest_availability_of_person_by_date[person].items():
            print(f"{person} on {k} is first free after {v}")
                
        return earliest_availability_of_person_by_date
        
    def get_all_earliest_availability(self):
        time_slots = defaultdict(list)
        for person in self.schedules:
            for date in self.schedules[person]:
                for activity in date['activities']:
                    time_slots[date['date']].append(activity)
                    
        # print(time_slots)
        busy_times_by_date = defaultdict(list)
        for k,v in time_slots.items():
            merged = self.merge_intervals(intervals=v)
            # print((merged))
            for activity in merged:
                busy_times_by_date[k].append([activity['start'],activity['end']])
                # print(k,activity['start'],activity['end'])
            print(f"Earliest time on {k} is {merged[0]['end']}")
            
        print(busy_times_by_date)
        return busy_times_by_date

    def visualize_calendar(self):
        """Plots the schedules of all people in a Gantt chart format for each day
        """
        gantt_charts = defaultdict(list)
        for person in self.schedules:
            for date in self.schedules[person]: 
                for activity in date['activities']:
                    gantt_charts[date['date']].append(dict(Task=f"{person}",Start=activity['start'],Finish=activity['end'],Person=f"{person}",Activity=activity['activity']))
        
        for date in gantt_charts:
            df = pd.DataFrame(gantt_charts[date])
            fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", 
            color="Person",title=f"{date}",hover_data=['Activity'])

            fig.update_layout(xaxis=dict(
                            title=f'{date}', 
                            # tickformat = '%S'
                            ))
            fig.update_yaxes(autorange="reversed")
            fig.show()
        

    
def main():
    schedules = SceduleGenerator(start_year=2020, end_year=2020, n_persons=5, n_days=2, min_activites=5, max_activites=10).schedules
    my_cal = MyCal(schedules=schedules)
    for person in my_cal.persons:
        my_cal.get_earliest_availability(person=person)
        
    my_cal.get_all_earliest_availability()
    my_cal.visualize_calendar()
        
        
if __name__ == "__main__":   
    main()

Tristan Ingram on 2020-01-21 is first free after 2020-01-21 02:19:29.419626
Tristan Ingram on 2020-01-22 is first free after 2020-01-22 08:09:30.921953
Rex Levy on 2020-01-21 is first free after 2020-01-21 04:22:26.385936
Rex Levy on 2020-01-22 is first free after 2020-01-22 08:52:20.568455
Cortez Mclaughlin on 2020-01-21 is first free after 2020-01-21 02:10:37.397160
Cortez Mclaughlin on 2020-01-22 is first free after 2020-01-22 05:39:22.536454
Perry Mcneil on 2020-01-21 is first free after 2020-01-21 08:35:43.947230
Perry Mcneil on 2020-01-22 is first free after 2020-01-22 02:26:23.975051
Julee Jarvis on 2020-01-21 is first free after 2020-01-21 08:03:02.673214
Julee Jarvis on 2020-01-22 is first free after 2020-01-22 05:08:11.673735
Earliest time on 2020-01-21 is 2020-01-21 02:19:29.419626
Earliest time on 2020-01-22 is 2020-01-22 19:11:35.288968
defaultdict(<class 'list'>, {datetime.date(2020, 1, 21): [[datetime.datetime(2020, 1, 21, 1, 14, 37, 397160), datetime.datetime(2020, 1, 2